<a href="https://colab.research.google.com/github/uoahy-6uoas/HG-Machine/blob/main/hg5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

keywords = 검증세트, 교차검증, 그리드서치, 랜덤서치

test세트에 맞추다 보면 결국엔 test에만 잘 맞는 모델이 만들어짐.. 따라서 검증세트가 필요<br>
<br>
**검증세트**<br>
> 훈련 세트를 사용해 다양한 하이퍼파라미터로 여러 모델을 훈련시키고 검증 세트에서 최상의 성능을 내는 모델과 하이퍼파라미터를 선택합니다. *텐서플로우블로그*

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy() # ()잊지말자

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2)

In [4]:
#검증세트는 훈련세트에서 따로 뽑아 씀
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2)

In [5]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
dt.score(sub_input, sub_target), dt.score(test_input,test_target)

(0.997834977146981, 0.8676923076923077)

**교차검증**
> 데이터를 여러 번 반복해서 나누고 여러 모델을 학습하여 성능을 평가하는 방법 = 검증세트를 여러(k)번 나눠 따로 학습(k-fold)  

In [6]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
scores #모델 훈련하는 시간, 검증하는 시간, 점수, 기본적으로 5-fold 교차검증

{'fit_time': array([0.02037501, 0.01604128, 0.01105475, 0.01130676, 0.01276922]),
 'score_time': array([0.00162268, 0.00164962, 0.00166988, 0.00168276, 0.00173759]),
 'test_score': array([0.87307692, 0.85096154, 0.84119346, 0.84889317, 0.86236766])}

In [7]:
#교차검증의 최종점수
np.mean(scores['test_score'])

0.85529854890057

In [8]:
#train_test_split으로 섞지 않고 섞는 방법
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.8576088324572444

In [9]:
#10-fold-cross-validate
splitter = StratifiedKFold(n_splits=10,shuffle=True)
scores = cross_validate(dt,train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8666555506150881

모델파라미터 = 머신러닝 모델이 학습하는 파라미터 vs. 하이퍼파라미터 = 사용자가 지정, 자동으로 하이퍼파라미터 튜닝 : AutoML<br>
**그리드 서치**
> GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차검증 수행

In [10]:
#min_impurity_decrease 의 최적값을 찾아보자
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
# min_impurity_decrease : (정보이득*(노드샘플수)/(전체샘플수) <= 0.0005면 더 이상 분할 하지 않음)

In [11]:
#최적의 하이퍼파라미터를 찾아보자
gs = GridSearchCV(DecisionTreeClassifier(),params,n_jobs=-1)#n_jobs는 사용할 cpu개수, 기본값=1, -1이면 있는거 다씀
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [12]:
#찾았으면 그 찾은 값으로 전체훈련세트로 모델을 다시 만들어야함
#그리드서치는 찾은다음 자동으로 최적 파라미터로 훈련함
dt = gs.best_estimator_ #dt에 최적모델(best_estimator_) 저장함 근데 모델은 타입이 뭐지..?
type(dt)

sklearn.tree._classes.DecisionTreeClassifier

In [13]:
gs.best_params_ #찾아온 최적 값이 뭔지

{'min_impurity_decrease': 0.0002}

In [14]:
#각 매개변수에서 훈련한 점수 평균값
gs.cv_results_['mean_test_score'] #0.0003(gs.best_params_)이 높은 것을 확인

array([0.86299511, 0.86953894, 0.86934719, 0.86395758, 0.86472681])

In [15]:
#np.argmax
best_idx = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_idx]

{'min_impurity_decrease': 0.0002}

In [16]:
#여러 파라미터 실습
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), #range와 달리 float도 가능!
          'max_depth':range(5,20,1), #트리 깊이 제한
          'min_samples_split':range(2,100,10) # 노드를 나눌 최소 샘플 수
          }

In [17]:
gs = GridSearchCV(DecisionTreeClassifier(),params,n_jobs=-1)
gs.fit(train_input, train_target)
gs.best_params_

{'max_depth': 14,
 'min_impurity_decrease': 0.00030000000000000003,
 'min_samples_split': 12}

**랜덤서치**<br>
> 임의의 하이퍼파라미터를 선정하는 과정을 통해 최적의 해를 찾아가는 기법 *heytech_tstory*

In [18]:
#scipy lib
from scipy.stats import uniform, randint #주어진 범위에서 고르게 값(각각 : float,int)을 뽑음

In [19]:
rgen = randint(0,10)
rgen.rvs(10) # pop 10

array([4, 9, 8, 6, 0, 0, 8, 0, 4, 9])

In [20]:
np.unique(rgen.rvs(1000), return_counts=True) #카운트함수

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([114, 108,  93,  93,  74, 112, 103,  91, 107, 105]))

In [21]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.03717279, 0.44365756, 0.46241459, 0.59318496, 0.2794842 ,
       0.76344148, 0.681214  , 0.27455445, 0.97320089, 0.95651615])

In [22]:
#랜덤서치
params = {'min_impurity_decrease' : uniform(0,1),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf':randint(1,25) #min_samples_leaf : 말단(뿌리)노드가 되기 위한 최소 샘플 개수
          }


from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(), params,n_iter=100, n_jobs=-1)
gs.fit(train_input, train_target)
gs.best_params_, np.max(gs.cv_results_['mean_test_score'])

({'max_depth': 48,
  'min_impurity_decrease': 0.00958362020839909,
  'min_samples_leaf': 13,
  'min_samples_split': 6},
 0.8451029096024284)

In [23]:
dt = gs.best_estimator_
dt.score(test_input,test_target)
#얘는 아니지만 일반적으로 검증점수보다 테스트점수가 낮음

0.8453846153846154

* 확인문제
1. 훈련세트를 여러 개의 폴드로 나누고 폴드 1개는 평가용도로, 나머지 폴드는 훈련용으로 사용하는데, <br>그다음 모든 폴드를 평가용도로 사용하게끔 폴드 개수만큼 이 과정을 반복하는 평가방법은 교차검증 이라고 한다.
2. 교차검증을 수행하지 않는 함수느 클래스는 train_test_split
3. 마지막 RandomizedSearchCV예제에서 splitter='random'변수를 추가하고 훈련해보자, splitter기본 값인 best와 차이점은?

In [24]:
import time
i = time.time()
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random'), params,n_iter=100, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_, np.max(gs.cv_results_['mean_test_score']))
#random이면 무작위로 노드 분할하고 가장 좋은 것을 고름
f = time.time()
print(f-i)

{'max_depth': 49, 'min_impurity_decrease': 0.02763621295232499, 'min_samples_leaf': 19, 'min_samples_split': 19} 0.770831420744799
1.0603928565979004


In [25]:
i = time.time()
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='best'), params,n_iter=100, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_, np.max(gs.cv_results_['mean_test_score']))
f = time.time()
print(f-i) #차이점은 시간이 적게 걸리지만 점수가 낮다?

{'max_depth': 20, 'min_impurity_decrease': 0.0011639787396302559, 'min_samples_leaf': 9, 'min_samples_split': 12} 0.8579914118605168
1.2958781719207764
